# Identify tags in airline database

## Pending to complete the data creation

# Alternative solution

## Architecture
    - Two inputs: 
        previous words seq_inputf = [W_i-k, ..., W_i-1, W_i] 
        Next words seq_inputb = [W_i, W_i+1, ...,W_i+k]
    - Two embeding layers with shared weights
    - Two LSTM, one forward and one backward
    - A final concatenation and a dense layer

In [ ]:
# Use tensorflow 2.0

#CPU
#!pip install -q tensorflow==2.0.0

#GPU
#!pip install -q tensorflow-gpu==2.0.0


In [1]:
from __future__ import print_function

import sys
import os 
import numpy as np 
import tensorflow as tf 
print(tf.__version__)

#Show images
import matplotlib.pyplot as plt
%matplotlib inline
# plt configuration
plt.rcParams['figure.figsize'] = (10, 10)


1.14.0


In [3]:
# download data
#! wget https://s3-eu-west-1.amazonaws.com/text-mining-course/atis.zip
#! unzip atis.zip

# Read data
import pickle
import sys

atis_file = './atis/atis.pkl'

with open(atis_file,'rb') as f:
    if sys.version_info.major==2:
        train, test, dicts = pickle.load(f) #python2.7
    else:
        train, test, dicts = pickle.load(f, encoding='bytes') #python3

#Dictionaries and train test partition
w2idx, ne2idx, labels2idx = dicts[b'words2idx'], dicts[b'tables2idx'], dicts[b'labels2idx']
    
idx2w  = dict((v,k) for k,v in w2idx.items())
idx2la = dict((v,k) for k,v in labels2idx.items())

train_x, _, train_label = train
test_x,  _,  test_label  = test


# Max value of word coding to assign the ID_PAD
ID_PAD = np.max([np.max(tx) for tx in train_x]) + 1
print('ID_PAD: ', ID_PAD)

ID_PAD:  572


In [22]:

def context(l, size=3):
    l = list(l)
    lpadded = size * [ID_PAD] + l + size * [ID_PAD]
    out_f = [lpadded[(i - size):i+1] for i in range(size, size + len(l), 1)]
    out_b = [lpadded[i:(i + size)+1] for i in range(size, size + len(l), 1)]
    return out_f, out_b

context([1,2,3], size=3)

([[572, 572, 572, 1], [572, 572, 1, 2], [572, 1, 2, 3]],
 [[1, 2, 3, 572], [2, 3, 572, 572], [3, 572, 572, 572]])

In [26]:
def context(l, size=3):
    l = list(l)
    lpadded = size * [ID_PAD] + l + size * [ID_PAD]
    out_f = [lpadded[(i - size):i+1] for i in range(size, size + len(l), 1)]
    out_b = [lpadded[i:(i + size)+1] for i in range(size, size + len(l), 1)]
    return out_f, out_b


input_seq_length = 11

# Create train and test X y.
X_trn_f = []
X_trn_b = []
for s in train_x:
    out_f, out_b = context(s, size=input_seq_length-1)
    X_trn_f += out_f
    X_trn_b += out_b
X_trn_f = np.array(X_trn_f)
X_trn_b = np.array(X_trn_b)

X_tst_f = []
X_tst_b = []
for s in test_x:
    out_f, out_b = context(s,size=10)
    X_tst_f += out_f
    X_tst_b += out_b
X_tst_f = np.array(X_tst_f)
X_tst_b = np.array(X_tst_b)

print('X trn shape: ', X_trn_f.shape, X_trn_b.shape)
print('X_tst shape: ', X_tst_f.shape, X_tst_b.shape)


y_trn=[]
for s in train_label:
    y_trn += list(s)
y_trn = np.array(y_trn)
print('y_trn shape: ',y_trn.shape)

y_tst=[]
for s in test_label:
    y_tst += list(s)
y_tst = np.array(y_tst)
print('y_tst shape: ',y_tst.shape)


print('Num labels: ',len(set(y_trn)))
print('Num words: ',len(set(idx2w)))

X trn shape:  (56590, 11) (56590, 11)
X_tst shape:  (9198, 11) (9198, 11)
y_trn shape:  (56590,)
y_tst shape:  (9198,)
Num labels:  121
Num words:  572


In [29]:
# data attributes
input_seq_length = X_trn_f.shape[1]
input_vocabulary_size = len(set(idx2w)) + 1
output_length = 127

#Model parameters
embedding_size=64
num_hidden_lstm = 128


In [31]:
# build the model: Simple LSTM with embedings
print('Build model 1')
seq_input_f = tf.keras.layers.Input(shape=([input_seq_length]), name='prev1') 
seq_input_b = tf.keras.layers.Input(shape=([input_seq_length]), name='prev2') 
    
embeds = tf.keras.layers.Embedding(input_vocabulary_size, embedding_size)

embeds_f = embeds(seq_input_f)
embeds_b = embeds(seq_input_b)

lstm_f = tf.keras.layers.LSTM(128)(embeds_f)
lstm_b = tf.keras.layers.LSTM(128, go_backwards=True)(embeds_b)

concat = tf.keras.layers.concatenate([lstm_f, lstm_b], axis=-1)

    
output = tf.keras.layers.Dense(output_length, activation='softmax')(concat)


model1 = tf.keras.models.Model(inputs=[seq_input_f, seq_input_b], outputs=output)
model1.summary()

# Optimizer
adam_optimizer = tf.keras.optimizers.Adam()
model1.compile(loss='sparse_categorical_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])


Build model 1
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
prev1 (InputLayer)              [(None, 11)]         0                                            
__________________________________________________________________________________________________
prev2 (InputLayer)              [(None, 11)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 11, 64)       36672       prev1[0][0]                      
                                                                 prev2[0][0]                      
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 128)          98816       embedding_1[0][

In [34]:
#Fit model

tb_callback = tf.keras.callbacks.TensorBoard(log_dir='./tensorboard/airline/shared_embedings/')

history = model1.fit([X_trn_f, X_trn_b], y_trn, batch_size=128, epochs=20,
           validation_data=([X_tst_f, X_tst_b], y_tst), callbacks=[tb_callback])


W1029 18:24:34.806710 4616394176 deprecation.py:323] From /Users/jorge/anaconda3/envs/bankia/lib/python3.5/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 56590 samples, validate on 9198 samples
Epoch 1/20
56590/56590 [==============================] - 19s 327us/sample - loss: 1.2284 - acc: 0.7508 - val_loss: 0.5949 - val_acc: 0.8845
Epoch 2/20
56590/56590 [==============================] - 18s 322us/sample - loss: 0.2816 - acc: 0.9412 - val_loss: 0.2350 - val_acc: 0.9499
Epoch 3/20
56590/56590 [==============================] - 19s 336us/sample - loss: 0.1168 - acc: 0.9766 - val_loss: 0.1669 - val_acc: 0.9697
Epoch 4/20
56590/56590 [==============================] - 19s 337us/sample - loss: 0.0687 - acc: 0.9861 - val_loss: 0.1509 - val_acc: 0.9736
Epoch 5/20
56590/56590 [==============================] - 19s 330us/sample - loss: 0.0475 - acc: 0.9900 - val_loss: 0.1269 - val_acc: 0.9765
Epoch 6/20
56590/56590 [==============================] - 19s 328us/sample - loss: 0.0356 - acc: 0.9920 - val_loss: 0.1230 - val_acc: 0.9770
Epoch 7/20
56590/56590 [==============================] - 19s 328us/sample - loss: 0.0269 - acc: 0.9937 -